In [ ]:
#!pip install ray[rllib,tune]
#!pip install pettingzoo pygame pymunk
#!pip install torch

In [10]:
from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec
from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import PettingZooEnv, ParallelPettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

In [18]:
import sys
sys.argv = ['']

parser = add_rllib_example_script_args(
    default_iters=200,
    default_timesteps=1000000,
    default_reward=0.0,
)
args = parser.parse_args()

In [25]:
num_agents = 2

register_env(f"{num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_agents)))
policies = {f"pursuer_{i}" for i in range(num_agents)}

base_config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(
        #waterworld_v4,#.env(n_pursuers=args.num_agents)
        f"{args.num_agents}_agent_env",
        #env_config={"n_pursuers": args.num_agents},
    )
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        model_config_dict={"vf_share_layers": True},
        rl_module_spec=MultiAgentRLModuleSpec(
            module_specs={p: SingleAgentRLModuleSpec() for p in policies},
        ),
    )
)

In [26]:
results = run_rllib_example_script_experiment(base_config, args)

RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.